In [25]:
import tkinter as tk
from tkinter import Menu
from PIL import ImageTk, Image
import os
from tkinter import ttk  # สำหรับ Combobox
from tkinter import messagebox

# === หน้าต่างหลัก ===
window = tk.Tk()
window.title("My program")
window.geometry("900x600")

# === Toolbar ===
toolbar = tk.Frame(window, height=40, bg="#e0e0e0")
toolbar.pack(side="top", fill="x")

play_button = tk.Button(toolbar, text="▶", width=3)
play_button.pack(side="left", padx=5, pady=5)

reset_button = tk.Button(toolbar, text="⏹", width=3)
reset_button.pack(side="left", padx=5, pady=5)

clear_button = tk.Button(toolbar, text="⏺", width=3)
clear_button.pack(side="left", padx=5, pady=5)


# === แถบด้านซ้าย ===
left_panel = tk.Frame(window, bg='#2e2e2e', width=200)
left_panel.pack(side='left', fill='y')

def add_item(text):
    item_frame = tk.Frame(left_panel, bg='white')
    item_frame.pack(fill='x', pady=5, padx=10)
    button = tk.Button(item_frame, text=text, bg='white', relief='flat', command=lambda: item_clicked(text))
    button.pack(fill='x')

add_item("อุปกรณ์ 1")
add_item("อุปกรณ์ 2")
add_item("อุปกรณ์ 3")

# === Menu ===
menu = Menu(window)
file_menu = Menu(menu)
file_menu.add_command(label='New')
file_menu.add_separator()
file_menu.add_command(label='Save')
file_menu.add_separator()
file_menu.add_command(label='Save as')
menu.add_cascade(label='File', menu=file_menu)
menu.add_cascade(label='Edit', menu=file_menu)
menu.add_cascade(label='View', menu=file_menu)
menu.add_cascade(label='Code', menu=file_menu)
menu.add_cascade(label='Window', menu=file_menu)
menu.add_cascade(label='Help', menu=file_menu)
window.config(menu=menu)

# === Canvas ===
canvas = tk.Canvas(window, bg='#7a7576')
canvas.pack(expand=True, fill='both')

# === ตัวแปรหลัก ===
id_map = {}
slot_coords = []
slot_rects = []
image_slot_map = {}
drag_data = {"item": None, "x": 0, "y": 0}

# === โหลดภาพ ===
def load_image(image_path, size=(120, 80)):
    if os.path.exists(image_path):
        img = ImageTk.PhotoImage(Image.open(image_path).resize(size))
        id_map[str(id(img))] = os.path.basename(image_path)
        return img
    else:
        print(f"Error: {image_path} not found.")
        return None

img1_1 = load_image("D:/PythonProject/photo/อุปกรณ์ 1/9815-1.png")
img1_2 = load_image("D:/PythonProject/photo/อุปกรณ์ 1/o8.jpg")
img2_1 = load_image("D:/PythonProject/photo/อุปกรณ์ 2/Breadboard.png")
img2_2 = load_image("D:/PythonProject/photo/อุปกรณ์ 2/LED.png")
img3_1 = load_image("D:/PythonProject/photo/อุปกรณ์ 3/3.1.png")
img3_2 = load_image("D:/PythonProject/photo/อุปกรณ์ 3/3.2.png")
img3_3 = load_image("D:/PythonProject/photo/อุปกรณ์ 3/3.3.png")
img3_4 = load_image("D:/PythonProject/photo/อุปกรณ์ 3/3.4.png")



# === ช่อง 6 ช่อง ===
def update_slots():
    canvas.delete("slot")
    slot_coords.clear()
    slot_rects.clear()

    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()

    slot_width = 150
    slot_height = 100
    space_x = 60
    space_y = 50

    start_x = (canvas_width - (slot_width * 3 + space_x * 2)) // 2
    start_y = (canvas_height - (slot_height * 2 + space_y)) // 2 + 40

    for i in range(6):
        x1 = start_x + (i % 3) * (slot_width + space_x)
        y1 = start_y + (i // 3) * (slot_height + space_y)
        x2 = x1 + slot_width
        y2 = y1 + slot_height

        rect = canvas.create_rectangle(x1, y1, x2, y2, fill="lightgray", outline="black", width=2, tags="slot")
        slot_coords.append((x1, y1, x2, y2))
        slot_rects.append(rect)

# === ลากภาพ ===
def on_start_drag(event):
    item = canvas.find_closest(event.x, event.y)[0]
    drag_data["item"] = item
    drag_data["x"] = event.x
    drag_data["y"] = event.y

def on_drag(event):
    dx = event.x - drag_data["x"]
    dy = event.y - drag_data["y"]
    canvas.move(drag_data["item"], dx, dy)
    drag_data["x"] = event.x
    drag_data["y"] = event.y

def on_stop_drag(event):
    if drag_data["item"] is None:
        return

    x, y = event.x, event.y
    image_id = drag_data["item"]
    image_bbox = canvas.bbox(image_id)

    tags = canvas.gettags(image_id)
    image_tag = tags[1] if len(tags) > 1 else None

    for i, (x1, y1, x2, y2) in enumerate(slot_coords):
        if x1 <= x <= x2 and y1 <= y <= y2:
            img_width = image_bbox[2] - image_bbox[0]
            img_height = image_bbox[3] - image_bbox[1]
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            canvas.coords(image_id, center_x - img_width // 2, center_y - img_height // 2)

            image_slot_map[image_id] = i
            break

    drag_data["item"] = None

# === ลบภาพด้วยคลิกขวา ===
def delete_image(event):
    item = canvas.find_closest(event.x, event.y)[0]
    canvas.delete(item)

# === วางภาพบน Canvas ===
def show_on_canvas(image):
    img_id = canvas.create_image(100, 100, image=image, anchor="nw", tags=("draggable", str(id(image))))
    canvas.tag_bind(img_id, "<Button-1>", on_start_drag)
    canvas.tag_bind(img_id, "<B1-Motion>", on_drag)
    canvas.tag_bind(img_id, "<ButtonRelease-1>", on_stop_drag)
    canvas.tag_bind(img_id, "<Button-3>", delete_image)
    return img_id

# === ตรวจสอบเมื่อกด ▶ ===
def check_slots():
    correct_images = {"9815-1.png", "Breadboard.png", "LED.png"}
    found_images = set()
    correct_count = 0
    total_placed = 0

    # รีเซ็ตสีช่องก่อน
    for i, rect in enumerate(slot_rects):
        canvas.itemconfig(rect, fill="lightgray")

    for image_id, slot_index in image_slot_map.items():
        tags = canvas.gettags(image_id)
        if len(tags) < 2:
            continue

        image_tag = tags[1]
        image_name = id_map.get(image_tag, "")
        total_placed += 1

        if image_name in correct_images:
            found_images.add(image_name)
            correct_count += 1
            canvas.itemconfig(slot_rects[slot_index], fill="green")
        else:
            canvas.itemconfig(slot_rects[slot_index], fill="red")

    if correct_count == 0:
        return  # ยังไม่มีอุปกรณ์ถูกเลย

    if found_images == correct_images and total_placed == 3:
        messagebox.showinfo("ผลลัพธ์", "ถูกต้อง")
    elif found_images.issubset(correct_images) and total_placed <= 3:
        messagebox.showwarning("ผลลัพธ์", "วางอุปกรณ์ไม่ครบ")
    else:
        messagebox.showwarning("ผลลัพธ์", "ยังไม่ถูกต้อง")


# === รีเซ็ตสีเมื่อกด ⏹ ===
def reset_slots():
    for rect in slot_rects:
        canvas.itemconfig(rect, fill="lightgray")

# === ล้างทั้งหมดเมื่อกด ⏺ ===
def clear_all():
    canvas.delete("draggable")
    for rect in slot_rects:
        canvas.itemconfig(rect, fill="lightgray")
    image_slot_map.clear()

# === popup ภาพอุปกรณ์ ===
def item_clicked(item):
    popup = tk.Toplevel()
    popup.title(f"รายละเอียด {item}")
    popup.geometry("450x500")
    popup.configure(bg="white")

    tk.Label(popup, text=item, font=("TH Sarabun New", 18, "bold"), bg="white").pack(pady=10)
    if item == "อุปกรณ์ 1":
        if img1_1:
            tk.Button(popup, image=img1_1, bg="white", command=lambda: show_on_canvas(img1_1)).pack(pady=(5, 0))
            tk.Label(popup, text="Arduino UNO", bg="white", font=("TH Sarabun New", 12)).pack()
        if img1_2:
            tk.Button(popup, image=img1_2, bg="white", command=lambda: show_on_canvas(img1_2)).pack(pady=(10, 0))
            tk.Label(popup, text="Arduino ESP32", bg="white", font=("TH Sarabun New", 12)).pack()
    elif item == "อุปกรณ์ 2":
        if img2_1:
            tk.Button(popup, image=img2_1, bg="white", command=lambda: show_on_canvas(img2_1)).pack(pady=(5, 0))
            tk.Label(popup, text="Breadboard", bg="white", font=("TH Sarabun New", 12)).pack()
        if img2_2:
            tk.Button(popup, image=img2_2, bg="white", command=lambda: show_on_canvas(img2_2)).pack(pady=(5, 0))
            tk.Label(popup, text="LED", bg="white", font=("TH Sarabun New", 12)).pack()
    elif item == "อุปกรณ์ 3":
        if img3_1:
            tk.Button(popup, image=img3_1, bg="white", command=lambda: show_on_canvas(img3_1)).pack(pady=(5, 0))
            tk.Label(popup, text="Ultrasonic", bg="white", font=("TH Sarabun New", 12)).pack()
        if img3_2:
            tk.Button(popup, image=img3_2, bg="white", command=lambda: show_on_canvas(img3_2)).pack(pady=(5, 0))
            tk.Label(popup, text="1.5 V Battery", bg="white", font=("TH Sarabun New", 12)).pack()
        if img3_3:
            tk.Button(popup, image=img3_3, bg="white", command=lambda: show_on_canvas(img3_3)).pack(pady=(5, 0))
            tk.Label(popup, text="temperature Sensor", bg="white", font=("TH Sarabun New", 12)).pack()
        if img3_4:
            tk.Button(popup, image=img3_4, bg="white", command=lambda: show_on_canvas(img3_4)).pack(pady=(5, 0))
            tk.Label(popup, text="Micro Servo", bg="white", font=("TH Sarabun New", 12)).pack()



# === คำถามหลายข้อ ===
questions = {
    "ข้อ 1": "จงทำให้ไฟLEDติดโดยใช้วิธีที่ง่ายที่สุด",
    "ข้อ 2": "ยังไม่ได้คิดโจทย์จ้า",
    "ข้อ 3": "ยังไม่ได้คิดโจทย์จ้า",
    "ข้อ 4": "ยังไม่ได้คิดโจทย์จ้า"
}

solutions = {
    "ข้อ 1": ["Arduino UNO", "Breadboard", "LED"],
    "ข้อ 2": [""],
    "ข้อ 3": [],
    "ข้อ 4": []
}

def show_solution():
    current = selected_question.get()
    answer = solutions.get(current, [])

    if not answer:
        messagebox.showinfo("เฉลย", "ข้อนี้ยังไม่มีเฉลย")
    else:
        message = "อุปกรณ์ที่ต้องใช้:\n"
        for file_name in answer:
            display_name = image_names.get(file_name, file_name)
            message += f"- {display_name}\n"
        messagebox.showinfo("เฉลย", message)


# ===ฟังชั่นเรียกใช้ปุ่มเฉลย===
def show_solution():
    current = selected_question.get()
    answer = solutions.get(current, [])
    if not answer:
        messagebox.showinfo("เฉลย", "ข้อนี้ยังไม่มีเฉลย")
    else:
        message = "อุปกรณ์ที่ต้องใช้:\n" + "\n".join(f"- {name}" for name in answer)
        messagebox.showinfo("เฉลย", message)

# ===ปุ่มเฉลย===
solution_button = tk.Button(toolbar, text="💡", width=3, command=show_solution)
solution_button.pack(side="left", padx=5, pady=5)

selected_question = tk.StringVar()
question_text = tk.StringVar()
question_text.set(list(questions.values())[0])

def update_question(event=None):
    question_text.set(questions.get(selected_question.get(), "ไม่มีคำถาม"))

def setup_question_widgets():
    global question_frame
    question_frame = tk.Frame(canvas, bg="#7a7576")

    combo = ttk.Combobox(question_frame, values=list(questions.keys()), textvariable=selected_question, state="readonly", width=20)
    combo.current(0)
    combo.bind("<<ComboboxSelected>>", update_question)
    combo.pack(pady=2)

    question_label = tk.Label(question_frame, textvariable=question_text, font=("TH Sarabun New", 16, "bold"),
                              fg="white", bg="#7a7576", wraplength=500)
    question_label.pack()

    canvas.create_window(0, 0, window=question_frame, tags="top_text")

def center_question_widgets():
    canvas_width = canvas.winfo_width()
    canvas.coords("top_text", canvas_width // 2, 60)

def update_layout():
    update_slots()
    center_question_widgets()

def delayed_update_all(event=None):
    window.after(20, update_layout)

setup_question_widgets()

# === Event Bindings ===
play_button.config(command=check_slots)
reset_button.config(command=reset_slots)
clear_button.config(command=clear_all)

canvas.bind("<Configure>", delayed_update_all)

# === Start GUI ===
window.mainloop()

Error: D:/PythonProject/photo/อุปกรณ์ 1/9815-1.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 1/o8.jpg not found.
Error: D:/PythonProject/photo/อุปกรณ์ 2/Breadboard.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 2/LED.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 3/3.1.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 3/3.2.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 3/3.3.png not found.
Error: D:/PythonProject/photo/อุปกรณ์ 3/3.4.png not found.


In [41]:
import tkinter as tk
from tkinter import PhotoImage
import webbrowser

# variable to store images
# 0 = compressor
# 1 = evaporator
# 2 = expansion valve
# 3 = condenser
# 4 = dryer
top_image = -1
left_image = -1
right_top_image = -1
right_bottom_image = -1
bottom_image = -1


def center_window(win, width, height):
    screen_width = win.winfo_screenwidth()
    screen_height = win.winfo_screenheight()
    x = int((screen_width - width) / 2)
    y = int((screen_height - height) / 2)
    win.geometry(f"{width}x{height}+{x}+{y}")


window = tk.Tk()
window.title("AC System")
center_window(window, 1200, 800)


top_panel = tk.Frame(window, height=40, bg="#e0e0e0")
top_panel.pack(side="top", fill="x")


def clear_data():
    global top_image, left_image, right_top_image, right_bottom_image, bottom_image
    top_image = -1
    left_image = -1
    right_top_image = -1
    right_bottom_image = -1
    bottom_image = -1
    canvas1.delete("all")
    canvas2.delete("all")
    canvas3.delete("all")
    canvas4.delete("all")
    print("Data cleared.")


def check_answer():
    global top_image, left_image, right_top_image, right_bottom_image, bottom_image
    if top_image == 0:
        canvas1.config(highlightbackground="green", highlightthickness=2)
    else:
        canvas1.config(highlightbackground="red", highlightthickness=2)

    if left_image == 1:
        canvas2.config(highlightbackground="green", highlightthickness=2)
    else:
        canvas2.config(highlightbackground="red", highlightthickness=2)

    if right_top_image == 3:
        canvas3.config(highlightbackground="green", highlightthickness=2)
    else:
        canvas3.config(highlightbackground="red", highlightthickness=2)

    if right_bottom_image == 4:
        canvas5.config(highlightbackground="green", highlightthickness=2)
    else:
        canvas5.config(highlightbackground="red", highlightthickness=2)

    if bottom_image == 2:
        canvas4.config(highlightbackground="green", highlightthickness=2)
    else:
        canvas4.config(highlightbackground="red", highlightthickness=2)


new_button = tk.Button(top_panel, text="New", command=clear_data)
new_button.pack(side="left", padx=5, pady=5)

check_button = tk.Button(top_panel, text="Check", command=check_answer)
check_button.pack(side="left", padx=5, pady=5)


canvas_frame = tk.Frame(window, bg="white")
canvas_frame.pack(side="top", fill="both", expand=True, padx=10, pady=10)


# Create a 3x3 grid of canvases
def show_image(canvas, image_path, position, ans):
    canvas.delete("all")  # Clear previous image
    img = PhotoImage(file=image_path)
    canvas.image = img  # Keep a reference to avoid garbage collection
    canvas.create_image(0, 0, anchor="nw", image=img)

    if position == "top":
        global top_image
        top_image = ans
        canvas1.config(highlightbackground="white", highlightthickness=0)
    elif position == "left":
        global left_image
        left_image = ans
        canvas2.config(highlightbackground="white", highlightthickness=0)
    elif position == "right_top_image":
        global right_top_image
        right_top_image = ans
        canvas3.config(highlightbackground="white", highlightthickness=0)
    elif position == "right_bottom_image":
        global right_bottom_image
        right_bottom_image = ans
        canvas5.config(highlightbackground="white", highlightthickness=0)
    elif position == "bottom":
        global bottom_image
        bottom_image = ans
        canvas4.config(highlightbackground="white", highlightthickness=0)

    print(f"Clicked on canvas1, {position}: {ans}")


def on_canvas1_click(event):
    popup = tk.Menu(window, tearoff=0)
    popup.add_command(
        label="Compressor",
        command=lambda: show_image(canvas1, "compressor.png", "top", 0),
    )
    popup.add_command(
        label="Evaporator",
        command=lambda: show_image(canvas1, "evaporator.png", "top", 1),
    )
    popup.add_command(
        label="Expansion Valve",
        command=lambda: show_image(canvas1, "expasion.png", "top", 2),
    )
    popup.add_command(
        label="Condenser",
        command=lambda: show_image(canvas1, "condensor.png", "top", 3),
    )
    popup.add_command(
        label="Dryer", command=lambda: show_image(canvas1, "dryer.png", "top", 4)
    )
    popup.post(event.x_root, event.y_root)


def on_canvas2_click(event):
    popup = tk.Menu(window, tearoff=0)
    popup.add_command(
        label="Compressor",
        command=lambda: show_image(canvas2, "compressor.png", "left", 0),
    )
    popup.add_command(
        label="Evaporator",
        command=lambda: show_image(canvas2, "evaporator.png", "left", 1),
    )
    popup.add_command(
        label="Expansion Valve",
        command=lambda: show_image(canvas2, "expasion.png", "left", 2),
    )
    popup.add_command(
        label="Condenser",
        command=lambda: show_image(canvas2, "condensor.png", "left", 3),
    )
    popup.add_command(
        label="Dryer", command=lambda: show_image(canvas2, "dryer.png", "left", 4)
    )
    popup.post(event.x_root, event.y_root)


def on_canvas3_click(event):
    popup = tk.Menu(window, tearoff=0)
    popup.add_command(
        label="Compressor",
        command=lambda: show_image(canvas3, "compressor.png", "right_top_image", 0),
    )
    popup.add_command(
        label="Evaporator",
        command=lambda: show_image(canvas3, "evaporator.png", "right_top_image", 1),
    )
    popup.add_command(
        label="Expansion Valve",
        command=lambda: show_image(canvas3, "expasion.png", "right_top_image", 2),
    )
    popup.add_command(
        label="Condenser",
        command=lambda: show_image(canvas3, "condensor.png", "right_top_image", 3),
    )
    popup.add_command(
        label="Dryer",
        command=lambda: show_image(canvas3, "dryer.png", "right_top_image", 4),
    )
    popup.post(event.x_root, event.y_root)


def on_canvas4_click(event):
    popup = tk.Menu(window, tearoff=0)
    popup.add_command(
        label="Compressor",
        command=lambda: show_image(canvas4, "compressor.png", "bottom", 0),
    )
    popup.add_command(
        label="Evaporator",
        command=lambda: show_image(canvas4, "evaporator.png", "bottom", 1),
    )
    popup.add_command(
        label="Expansion Valve",
        command=lambda: show_image(canvas4, "expasion.png", "bottom", 2),
    )
    popup.add_command(
        label="Condenser",
        command=lambda: show_image(canvas4, "condensor.png", "bottom", 3),
    )
    popup.add_command(
        label="Dryer", command=lambda: show_image(canvas4, "dryer.png", "bottom", 4)
    )
    popup.post(event.x_root, event.y_root)


def on_canvas5_click(event):
    popup = tk.Menu(window, tearoff=0)
    popup.add_command(
        label="Compressor",
        command=lambda: show_image(canvas5, "compressor.png", "right_bottom_image", 0),
    )
    popup.add_command(
        label="Evaporator",
        command=lambda: show_image(canvas5, "evaporator.png", "right_bottom_image", 1),
    )
    popup.add_command(
        label="Expansion Valve",
        command=lambda: show_image(canvas5, "expasion.png", "right_bottom_image", 2),
    )
    popup.add_command(
        label="Condenser",
        command=lambda: show_image(canvas5, "condensor.png", "right_bottom_image", 3),
    )
    popup.add_command(
        label="Dryer",
        command=lambda: show_image(canvas5, "dryer.png", "right_bottom_image", 4),
    )
    popup.post(event.x_root, event.y_root)


canvas1 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas1.grid(row=0, column=1, padx=10, pady=10)
canvas1.bind("<Button-1>", on_canvas1_click)

canvas2 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas2.grid(row=2, column=0, padx=10, pady=10)
canvas2.bind("<Button-1>", on_canvas2_click)

canvas3 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas3.grid(row=1, column=2, padx=10, pady=10)
canvas3.bind("<Button-1>", on_canvas3_click)

canvas4 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas4.grid(row=4, column=1, padx=10, pady=10)
canvas4.bind("<Button-1>", on_canvas4_click)

canvas5 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas5.grid(row=3, column=2, padx=10, pady=10)
canvas5.bind("<Button-1>", on_canvas5_click)

canvas6 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas6.grid(row=0, column=0)
canvas6.image = PhotoImage(file="1.png")
canvas6.create_image(0, 0, anchor="nw", image=canvas6.image)
canvas6.config(highlightbackground="white", highlightthickness=0)

canvas7 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas7.grid(row=0, column=2)
canvas7.image = PhotoImage(file="2.png")
canvas7.create_image(0, 0, anchor="nw", image=canvas7.image)
canvas7.config(highlightbackground="white", highlightthickness=0)

canvas8 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas8.grid(row=4, column=0)
canvas8.image = PhotoImage(file="5.png")
canvas8.create_image(0, 0, anchor="nw", image=canvas8.image)
canvas8.config(highlightbackground="white", highlightthickness=0)

canvas9 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas9.grid(row=4, column=2)
canvas9.image = PhotoImage(file="4.png")
canvas9.create_image(0, 0, anchor="nw", image=canvas9.image)
canvas9.config(highlightbackground="white", highlightthickness=0)

canvas10 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas10.grid(row=2, column=2)
canvas10.image = PhotoImage(file="3.png")
canvas10.create_image(0, 0, anchor="nw", image=canvas10.image)
canvas10.config(highlightbackground="white", highlightthickness=0)

canvas11 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas11.grid(row=3, column=0)
canvas11.image = PhotoImage(file="6.png")
canvas11.create_image(0, 0, anchor="nw", image=canvas11.image)
canvas11.config(highlightbackground="white", highlightthickness=0)

canvas12 = tk.Canvas(canvas_frame, bg="white", width=200, height=100)
canvas12.grid(row=1, column=0)
canvas12.image = PhotoImage(file="7.png")
canvas12.create_image(0, 0, anchor="nw", image=canvas12.image)
canvas12.config(highlightbackground="white", highlightthickness=0)

# Configure the grid to expand properly
canvas_frame.grid_rowconfigure(0, weight=1)
canvas_frame.grid_rowconfigure(1, weight=1)
canvas_frame.grid_rowconfigure(2, weight=1)
canvas_frame.grid_columnconfigure(0, weight=1)
canvas_frame.grid_columnconfigure(1, weight=1)
canvas_frame.grid_columnconfigure(2, weight=1)

window2 = None  # Initialize window2 as None


def show_data(fram, window):
    for widget in fram.winfo_children():
        widget.destroy()

    if window == "Main":
        canvas_info = tk.Canvas(fram, bg="white", width=1000, height=600)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="14.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        link_button = tk.Button(
            fram,
            text="Link",
            font=("Arial", 16),
            bg="white",
            command=lambda: window2.after(0, lambda: window2.destroy()) or window.after(0, lambda: window.destroy()) or webbrowser.open("https://youtu.be/zUiWd5gopmw?si=dg8C6j40gFQVmFV7"),
        )
        link_button.grid(row=1, column=0, pady=10)

    if window == "Evaporator":
        canvas_info = tk.Canvas(fram, bg="white", width=200, height=100)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="evaporator.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        label = tk.Label(
            fram, text="Evaporator Information", font=("Arial", 16), bg="white"
        )
        label.grid(row=1, column=0, pady=10)

        text = tk.Text(fram, wrap="word", width=50, height=20)
        text.grid(row=2, column=0, padx=10, pady=10)
        text.insert(
            "1.0",
            "Evaporator : สารทำความเย็นในรูปของละอองฝอยที่ไหลผ่านในตัว Evaporator จะดูดซับความร้อนภายในรถ ทำให้อากาศภายในรถเย็นขึ้น และเกิดกระบวนการกลายเป็นไออย่างทันทีทันใด ทำให้สารทำความเย็นกลับมาอยู่ในสถานะก๊าซอีกครั้ง และถูกส่งกลับมายัง Compressor",
        )

    if window == "Compressor":
        canvas_info = tk.Canvas(fram, bg="white", width=200, height=100)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="compressor.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        label = tk.Label(
            fram, text="Compressor Information", font=("Arial", 16), bg="white"
        )
        label.grid(row=1, column=0, pady=10)

        text = tk.Text(fram, wrap="word", width=50, height=20)
        text.grid(row=2, column=0, padx=10, pady=10)
        text.insert(
            "1.0",
            "Compressor : ทำหน้าที่ ดูดสารทำความเย็นที่มาจากอีวาปอเรเตอร์ และทำการอัดให้กลายเป็นสารทำความเย็นที่สถานะก๊าซ จากนั้นจะส่งผ่านไปยัง Condenser",
        )

    if window == "Condenser":
        canvas_info = tk.Canvas(fram, bg="white", width=200, height=100)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="condensor.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        label = tk.Label(
            fram, text="Condenser Information", font=("Arial", 16), bg="white"
        )
        label.grid(row=1, column=0, pady=10)

        text = tk.Text(fram, wrap="word", width=50, height=20)
        text.grid(row=2, column=0, padx=10, pady=10)
        text.insert(
            "1.0",
            "Condenser : ทำหน้าเปลี่ยนสารทำความเย็นในสถานะก๊าซให้กลายเป็นสถานะของเหลวผ่านการควบแน่น จากนั้นส่งต่อไปยัง Receiver Dryer",
        )

    if window == "Dryer":
        canvas_info = tk.Canvas(fram, bg="white", width=200, height=100)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="dryer.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        label = tk.Label(
            fram, text="Expansion Valve Information", font=("Arial", 16), bg="white"
        )
        label.grid(row=1, column=0, pady=10)

        text = tk.Text(fram, wrap="word", width=50, height=20)
        text.grid(row=2, column=0, padx=10, pady=10)
        text.insert(
            "1.0",
            "Receiver Dryer :  ทำหน้าที่ ดูดซับความชื้น กรองสิ่งสกปรกในระบบ และเป็นที่พักของสารทำความเย็น จากนั้นจะถูกส่งต่อไปยัง Expansion Valve",
        )

    if window == "Expansion":
        canvas_info = tk.Canvas(fram, bg="white", width=200, height=100)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="expasion.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

        label = tk.Label(
            fram, text="Expansion Valve Information", font=("Arial", 16), bg="white"
        )
        label.grid(row=1, column=0, pady=10)

        text = tk.Text(fram, wrap="word", width=50, height=20)
        text.grid(row=2, column=0, padx=10, pady=10)
        text.insert(
            "1.0",
            "Expansion Valve : ทำหน้าที่ ลดแรงดันของสารทำความเย็น และฉีดสารทำความเย็นในสถานะที่เป็นของเหลวในรูปแบบของฝอยละอองเข้าไปที่ Evaporator",
        )


def show_window2():
    global window2
    if window2 is None or not window2.winfo_exists():  # Check if window2 doesn't exist
        window2 = tk.Toplevel(window)
        window2.title("Info")
        center_window(window2, 1200, 800)
        # top_panel2 = tk.Frame(window2, height=40, bg="#e0e0e0")
        # top_panel2.pack(side="top", fill="x")
        # new_button2 = tk.Button(top_panel2, text="Main", command=window2.withdraw)
        # new_button2.pack(side="left", padx=5, pady=5)

        top_panel_info = tk.Frame(window2, height=40, bg="#e0e0e0")
        top_panel_info.pack(side="top", fill="x")

        canvas_frame2 = tk.Frame(window2, bg="white")
        canvas_frame2.pack(side="top", fill="both", expand=True, padx=10, pady=10)

        main_button = tk.Button(
            top_panel_info,
            text="Main",
            command=lambda: show_data(canvas_frame2, "Main"),
        )
        main_button.pack(side="left", padx=5, pady=5)

        compressor_button = tk.Button(
            top_panel_info,
            text="Compressor",
            command=lambda: show_data(canvas_frame2, "Compressor"),
        )

        compressor_button.pack(side="left", padx=5, pady=5)
        condenser_button = tk.Button(
            top_panel_info,
            text="Condenser",
            command=lambda: show_data(canvas_frame2, "Condenser"),
        )
        condenser_button.pack(side="left", padx=5, pady=5)

        dryer_button = tk.Button(
            top_panel_info,
            text="Dryer",
            command=lambda: show_data(canvas_frame2, "Dryer"),
        )
        dryer_button.pack(side="left", padx=5, pady=5)

        expansion_button = tk.Button(
            top_panel_info,
            text="Expansion",
            command=lambda: show_data(canvas_frame2, "Expansion"),
        )
        expansion_button.pack(side="left", padx=5, pady=5)

        link_button = tk.Button(
            top_panel_info,
            text="Link",
            command=lambda: webbrowser.open("https://youtu.be/zUiWd5gopmw?si=dg8C6j40gFQVmFV7"),
        )
        link_button.pack(side="left", padx=5, pady=5)

        canvas_info = tk.Canvas(canvas_frame2, bg="white", width=1000, height=600)
        canvas_info.grid(row=0, column=0)
        canvas_info.image = PhotoImage(file="14.png")
        canvas_info.create_image(0, 0, anchor="nw", image=canvas_info.image)
        canvas_info.config(highlightthickness=0)

    else:
        if window2.state() == "withdrawn":
            window2.deiconify()
        else:
            window2.withdraw()


show_button = tk.Button(top_panel, text="Show info", command=show_window2)
show_button.pack(side="left", padx=5, pady=5)

window.mainloop()